In [1]:
%%bash
docker container ls

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
99da1d19aa27        mongo               "docker-entrypoint.s…"   4 hours ago         Up 4 hours          0.0.0.0:27017->27017/tcp   dbms


In [2]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client.dbms
tweets = db.things
print("Done")

Done


In [31]:
import pprint
totalUser = [
    {"$group": {"_id": "$user"}},
    {"$count": "count"}]
pprint.pprint(list(tweets.aggregate(totalUser)))

[{'count': 659775}]


In [33]:
from bson.son import SON
mostActive = [
    {"$group": {"_id": "$user", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},
    {"$limit": 10}]
pprint.pprint(list(tweets.aggregate(mostActive)))

[{'_id': 'lost_dog', 'count': 549},
 {'_id': 'webwoke', 'count': 345},
 {'_id': 'tweetpet', 'count': 310},
 {'_id': 'SallytheShizzle', 'count': 281},
 {'_id': 'VioletsCRUK', 'count': 279},
 {'_id': 'mcraddictal', 'count': 276},
 {'_id': 'tsarnick', 'count': 248},
 {'_id': 'what_bugs_u', 'count': 246},
 {'_id': 'Karen230683', 'count': 238},
 {'_id': 'DarkPiano', 'count': 236}]


In [35]:
mostGrumpy = [
    {"$match": {"polarity": 0}},
    {"$group": {"_id": "$user", "user": {"$first": "$user"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},
    {"$project": {"_id": 0, "user": 1, "count": 1}},
    {"$limit": 5}]
pprint.pprint(list(tweets.aggregate(mostGrumpy)))

[{'count': 549, 'user': 'lost_dog'},
 {'count': 310, 'user': 'tweetpet'},
 {'count': 264, 'user': 'webwoke'},
 {'count': 210, 'user': 'mcraddictal'},
 {'count': 210, 'user': 'wowlew'}]


In [62]:
mostHappy = [
    {"$match": {"polarity": 4}},
    {"$group": {"_id": "$user", "user": {"$first": "$user"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},
    {"$project": {"_id": 0, "user": 1, "count": 1}},
    {"$limit": 1}]
pprint.pprint(list(tweets.aggregate(mostHappy)))

[{'count': 246, 'user': 'what_bugs_u'}]


In [103]:
mostLink = [
    {"$match": {"text": "/@\w+/"}},
    {"$group": {"_id": "$user", "user": {"$first": "$user"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},    
    {"$limit": 10},
    {"$project": {"_id": 0, "user": 1, "count": 1}}]
pprint.pprint(list(tweets.aggregate(mostLink)))

[]


In [116]:
mostMentioned = [
    {"$match": {"text": "/@\w+/"}},
    {"$project": {"user": 1, "text": {"$split": ["$text", " "]}}},
    {"$unwind": "$text"},
    {"$match": {"text": "/@\w+/"}},
    {"$project": {"user": 1, "text": {"$split": ["$text", "@"]}},
    {"$project": {"user": 1, "text": {"$arrayElemAt": ["$text", 1]}}}
    {"$group": {"_id": "$text", "user": {"$first": "$text"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},    
    {"$limit": 5},
    {"$project": {"_id": 0, "user": 1, "count": 1}}]
pprint.pprint(list(tweets.aggregate(mostMentioned)))

SyntaxError: invalid syntax (<ipython-input-116-f451df69dd3a>, line 8)